In [10]:
#Importing Libraries
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, MaxPool2D
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import Model
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.callbacks import Callback, ModelCheckpoint
import os
from glob import glob
import random
import cv2
import numpy as np
import sklearn
import scipy
from tqdm import tqdm
import skimage
from skimage.transform import resize
from sklearn import model_selection
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import train_test_split,KFold,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [11]:
# Loading Data
imageSize=50
trainDir = "../input/asl_alphabet_train/"
testDir =  "../input/asl_alphabet_test/"
def get_data(folder):
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['A']:
                label = 0
            elif folderName in ['B']:
                label = 1
            elif folderName in ['C']:
                label = 2
            elif folderName in ['D']:
                label = 3
            elif folderName in ['nothing']:
                label = 4
            else:
                label = 5
            for imageFileName in tqdm(os.listdir(folder + folderName)):
                imageFile = cv2.imread(folder + folderName + '/' + imageFileName)
                if imageFile is not None:
                    imageFile = skimage.transform.resize(imageFile, (imageSize, imageSize, 3), mode='constant')
                    img_arr = np.asarray(imageFile)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y
x_Train, y_Train = get_data(trainDir) 

x_Train, x_Test, y_Train, y_Test = train_test_split(x_Train, y_Train, test_size=0.2)

# Encode labels to vectors, this makes it easier for the algorithm to read the data in
y_TrainHot = to_categorical(y_Train, num_classes = 5)
y_TestHot = to_categorical(y_Test, num_classes = 5)

100%|██████████| 3000/3000 [00:12<00:00, 247.30it/s]


In [12]:
# Shuffle arrays or sparse matrices in a consistent way
x_Train, y_TrainHot = shuffle(x_Train, y_TrainHot, random_state=13)
x_Test, y_TestHot = shuffle(x_Test, y_TestHot, random_state=13)
x_Train = x_Train[:30000]
x_Test = x_Test[:30000]
y_TrainHot = y_TrainHot[:30000]
y_TestHot = y_TestHot[:30000]

In [13]:
class MetricsCheckpoint(Callback):
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

In [14]:
mapCharacters = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'nothing'}
classWeight = class_weight.compute_class_weight('balanced', np.unique(y_Train), y_Train)
weight_path1 = '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_model_1 = VGG16(weights = weight_path1, include_top=False, input_shape=(imageSize, imageSize, 3))
optimizer1 = keras.optimizers.Adam()
optimizer2 = keras.optimizers.RMSprop(lr=0.0001)
def pretrainedNetwork(xtrain,ytrain,xtest,ytest,pretrainedmodel,pretrainedweights,classweight,numclasses,numepochs,optimizer,labels):
    # Initially the model has no top
    base_model = pretrained_model_1 

    # Adding the Top layer
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(numclasses, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Training the top layer
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    model.summary()
    # Fit model
    history = model.fit(xtrain,ytrain, epochs=numepochs, class_weight=classweight, validation_data=(xtest,ytest), verbose=1,callbacks = [MetricsCheckpoint('logs')])
    # Evaluate model
    score = model.evaluate(xtest,ytest, verbose=0)
    print('\nKeras CNN - accuracy:', score[1], '\n')
    y_pred = model.predict(xtest)
    return model
pretrainedNetwork(x_Train, y_TrainHot, x_Test, y_TestHot,pretrained_model_1,weight_path1,classWeight,5,4,optimizer1,mapCharacters)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

 6528/12000 [===============>..............] - ETA: 10:57 - loss: 1.6989 - acc: 0.15 - ETA: 10:44 - loss: 1.6464 - acc: 0.17 - ETA: 10:09 - loss: 1.6426 - acc: 0.16 - ETA: 9:50 - loss: 1.6526 - acc: 0.1875 - ETA: 9:38 - loss: 1.6520 - acc: 0.200 - ETA: 9:29 - loss: 1.6485 - acc: 0.208 - ETA: 9:23 - loss: 1.6339 - acc: 0.232 - ETA: 9:21 - loss: 1.6249 - acc: 0.242 - ETA: 9:39 - loss: 1.6091 - acc: 0.256 - ETA: 9:55 - loss: 1.6028 - acc: 0.271 - ETA: 9:49 - loss: 1.5873 - acc: 0.289 - ETA: 9:43 - loss: 1.5799 - acc: 0.299 - ETA: 9:38 - loss: 1.5688 - acc: 0.305 - ETA: 9:32 - loss: 1.5581 - acc: 0.317 - ETA: 9:29 - loss: 1.5522 - acc: 0.320 - ETA: 9:25 - loss: 1.5551 - acc: 0.318 - ETA: 9:22 - loss: 1.5563 - acc: 0.318 - ETA: 9:19 - loss: 1.5460 - acc: 0.328 - ETA: 9:21 - loss: 1.5473 - acc: 0.319 - ETA: 9:24 - loss: 1.5490 - acc: 0.315 - ETA: 9:23 - loss: 1.5462 - acc: 0.318 - ETA: 9:25 - loss: 1.5396 - acc: 0.323 - ETA: 9:29 - loss: 1.5288 - acc: 0.339 - ETA: 9:34 - loss: 1.5218 - acc: 

12000/12000 [==============================] - ETA: 4:22 - loss: 0.9338 - acc: 0.745 - ETA: 4:21 - loss: 0.9316 - acc: 0.746 - ETA: 4:19 - loss: 0.9299 - acc: 0.747 - ETA: 4:18 - loss: 0.9278 - acc: 0.747 - ETA: 4:16 - loss: 0.9263 - acc: 0.748 - ETA: 4:14 - loss: 0.9250 - acc: 0.749 - ETA: 4:13 - loss: 0.9238 - acc: 0.749 - ETA: 4:11 - loss: 0.9220 - acc: 0.750 - ETA: 4:09 - loss: 0.9197 - acc: 0.751 - ETA: 4:08 - loss: 0.9177 - acc: 0.752 - ETA: 4:06 - loss: 0.9158 - acc: 0.753 - ETA: 4:05 - loss: 0.9139 - acc: 0.753 - ETA: 4:04 - loss: 0.9118 - acc: 0.754 - ETA: 4:02 - loss: 0.9103 - acc: 0.755 - ETA: 4:01 - loss: 0.9085 - acc: 0.756 - ETA: 4:00 - loss: 0.9070 - acc: 0.757 - ETA: 3:59 - loss: 0.9054 - acc: 0.758 - ETA: 3:57 - loss: 0.9036 - acc: 0.758 - ETA: 3:56 - loss: 0.9020 - acc: 0.759 - ETA: 3:54 - loss: 0.9004 - acc: 0.760 - ETA: 3:53 - loss: 0.8990 - acc: 0.760 - ETA: 3:52 - loss: 0.8982 - acc: 0.760 - ETA: 3:50 - loss: 0.8968 - acc: 0.761 - ETA: 3:48 - loss: 0.8952 - acc: 0

 6528/12000 [===============>..............] - ETA: 10:19 - loss: 0.3252 - acc: 1.00 - ETA: 11:16 - loss: 0.3950 - acc: 0.93 - ETA: 11:24 - loss: 0.4028 - acc: 0.94 - ETA: 11:09 - loss: 0.3803 - acc: 0.96 - ETA: 10:48 - loss: 0.3764 - acc: 0.96 - ETA: 10:34 - loss: 0.3612 - acc: 0.96 - ETA: 10:26 - loss: 0.3549 - acc: 0.96 - ETA: 10:21 - loss: 0.3549 - acc: 0.96 - ETA: 10:32 - loss: 0.3517 - acc: 0.97 - ETA: 10:31 - loss: 0.3523 - acc: 0.96 - ETA: 10:34 - loss: 0.3587 - acc: 0.96 - ETA: 10:28 - loss: 0.3598 - acc: 0.96 - ETA: 10:19 - loss: 0.3603 - acc: 0.96 - ETA: 10:14 - loss: 0.3689 - acc: 0.95 - ETA: 10:08 - loss: 0.3652 - acc: 0.95 - ETA: 10:01 - loss: 0.3607 - acc: 0.95 - ETA: 9:55 - loss: 0.3655 - acc: 0.9522 - ETA: 9:50 - loss: 0.3636 - acc: 0.954 - ETA: 9:45 - loss: 0.3666 - acc: 0.955 - ETA: 9:41 - loss: 0.3649 - acc: 0.954 - ETA: 9:40 - loss: 0.3619 - acc: 0.956 - ETA: 9:45 - loss: 0.3596 - acc: 0.958 - ETA: 9:47 - loss: 0.3625 - acc: 0.959 - ETA: 9:43 - loss: 0.3617 - acc: 

12000/12000 [==============================] - ETA: 4:37 - loss: 0.3165 - acc: 0.962 - ETA: 4:35 - loss: 0.3163 - acc: 0.962 - ETA: 4:33 - loss: 0.3162 - acc: 0.962 - ETA: 4:32 - loss: 0.3163 - acc: 0.962 - ETA: 4:30 - loss: 0.3163 - acc: 0.962 - ETA: 4:29 - loss: 0.3163 - acc: 0.961 - ETA: 4:27 - loss: 0.3161 - acc: 0.961 - ETA: 4:26 - loss: 0.3156 - acc: 0.961 - ETA: 4:24 - loss: 0.3154 - acc: 0.961 - ETA: 4:23 - loss: 0.3154 - acc: 0.961 - ETA: 4:21 - loss: 0.3149 - acc: 0.961 - ETA: 4:19 - loss: 0.3146 - acc: 0.962 - ETA: 4:18 - loss: 0.3145 - acc: 0.962 - ETA: 4:16 - loss: 0.3141 - acc: 0.961 - ETA: 4:15 - loss: 0.3142 - acc: 0.961 - ETA: 4:13 - loss: 0.3139 - acc: 0.961 - ETA: 4:11 - loss: 0.3133 - acc: 0.962 - ETA: 4:10 - loss: 0.3128 - acc: 0.962 - ETA: 4:08 - loss: 0.3131 - acc: 0.961 - ETA: 4:06 - loss: 0.3129 - acc: 0.962 - ETA: 4:04 - loss: 0.3129 - acc: 0.962 - ETA: 4:03 - loss: 0.3127 - acc: 0.962 - ETA: 4:02 - loss: 0.3127 - acc: 0.962 - ETA: 4:00 - loss: 0.3121 - acc: 0

 6528/12000 [===============>..............] - ETA: 8:51 - loss: 0.2507 - acc: 0.968 - ETA: 8:50 - loss: 0.2902 - acc: 0.937 - ETA: 9:19 - loss: 0.2687 - acc: 0.947 - ETA: 9:15 - loss: 0.2397 - acc: 0.953 - ETA: 9:07 - loss: 0.2510 - acc: 0.956 - ETA: 9:02 - loss: 0.2491 - acc: 0.958 - ETA: 8:57 - loss: 0.2428 - acc: 0.964 - ETA: 8:54 - loss: 0.2334 - acc: 0.964 - ETA: 8:50 - loss: 0.2339 - acc: 0.968 - ETA: 8:46 - loss: 0.2362 - acc: 0.968 - ETA: 8:45 - loss: 0.2357 - acc: 0.968 - ETA: 8:51 - loss: 0.2386 - acc: 0.971 - ETA: 8:50 - loss: 0.2356 - acc: 0.973 - ETA: 8:54 - loss: 0.2331 - acc: 0.973 - ETA: 8:52 - loss: 0.2335 - acc: 0.972 - ETA: 8:49 - loss: 0.2305 - acc: 0.972 - ETA: 8:47 - loss: 0.2300 - acc: 0.974 - ETA: 8:47 - loss: 0.2315 - acc: 0.974 - ETA: 8:44 - loss: 0.2302 - acc: 0.972 - ETA: 8:44 - loss: 0.2295 - acc: 0.973 - ETA: 8:44 - loss: 0.2323 - acc: 0.971 - ETA: 8:41 - loss: 0.2350 - acc: 0.970 - ETA: 8:40 - loss: 0.2341 - acc: 0.971 - ETA: 8:39 - loss: 0.2295 - acc: 0

12000/12000 [==============================] - ETA: 4:24 - loss: 0.1955 - acc: 0.977 - ETA: 4:22 - loss: 0.1957 - acc: 0.977 - ETA: 4:20 - loss: 0.1955 - acc: 0.977 - ETA: 4:19 - loss: 0.1954 - acc: 0.977 - ETA: 4:17 - loss: 0.1954 - acc: 0.977 - ETA: 4:16 - loss: 0.1956 - acc: 0.977 - ETA: 4:15 - loss: 0.1954 - acc: 0.977 - ETA: 4:13 - loss: 0.1956 - acc: 0.977 - ETA: 4:11 - loss: 0.1955 - acc: 0.977 - ETA: 4:10 - loss: 0.1952 - acc: 0.977 - ETA: 4:08 - loss: 0.1953 - acc: 0.977 - ETA: 4:07 - loss: 0.1952 - acc: 0.977 - ETA: 4:05 - loss: 0.1950 - acc: 0.977 - ETA: 4:03 - loss: 0.1948 - acc: 0.977 - ETA: 4:02 - loss: 0.1945 - acc: 0.977 - ETA: 4:00 - loss: 0.1944 - acc: 0.977 - ETA: 3:59 - loss: 0.1944 - acc: 0.977 - ETA: 3:57 - loss: 0.1949 - acc: 0.976 - ETA: 3:55 - loss: 0.1949 - acc: 0.976 - ETA: 3:54 - loss: 0.1949 - acc: 0.976 - ETA: 3:52 - loss: 0.1948 - acc: 0.976 - ETA: 3:50 - loss: 0.1948 - acc: 0.976 - ETA: 3:49 - loss: 0.1944 - acc: 0.976 - ETA: 3:47 - loss: 0.1942 - acc: 0

 6528/12000 [===============>..............] - ETA: 10:47 - loss: 0.1597 - acc: 0.96 - ETA: 12:22 - loss: 0.1272 - acc: 0.98 - ETA: 11:33 - loss: 0.1325 - acc: 0.97 - ETA: 10:56 - loss: 0.1337 - acc: 0.98 - ETA: 10:34 - loss: 0.1468 - acc: 0.98 - ETA: 10:17 - loss: 0.1470 - acc: 0.97 - ETA: 10:06 - loss: 0.1740 - acc: 0.96 - ETA: 10:30 - loss: 0.1681 - acc: 0.96 - ETA: 10:24 - loss: 0.1603 - acc: 0.97 - ETA: 10:16 - loss: 0.1616 - acc: 0.97 - ETA: 10:10 - loss: 0.1643 - acc: 0.97 - ETA: 10:08 - loss: 0.1573 - acc: 0.97 - ETA: 10:00 - loss: 0.1545 - acc: 0.97 - ETA: 9:52 - loss: 0.1535 - acc: 0.9799 - ETA: 9:48 - loss: 0.1528 - acc: 0.981 - ETA: 9:45 - loss: 0.1463 - acc: 0.982 - ETA: 9:38 - loss: 0.1474 - acc: 0.979 - ETA: 9:32 - loss: 0.1476 - acc: 0.979 - ETA: 9:29 - loss: 0.1455 - acc: 0.980 - ETA: 9:25 - loss: 0.1427 - acc: 0.981 - ETA: 9:20 - loss: 0.1440 - acc: 0.980 - ETA: 9:22 - loss: 0.1435 - acc: 0.981 - ETA: 9:22 - loss: 0.1438 - acc: 0.982 - ETA: 9:21 - loss: 0.1450 - acc: 

12000/12000 [==============================] - ETA: 4:18 - loss: 0.1419 - acc: 0.983 - ETA: 4:16 - loss: 0.1417 - acc: 0.983 - ETA: 4:15 - loss: 0.1413 - acc: 0.983 - ETA: 4:13 - loss: 0.1413 - acc: 0.983 - ETA: 4:12 - loss: 0.1413 - acc: 0.983 - ETA: 4:10 - loss: 0.1410 - acc: 0.983 - ETA: 4:09 - loss: 0.1410 - acc: 0.983 - ETA: 4:07 - loss: 0.1412 - acc: 0.983 - ETA: 4:06 - loss: 0.1413 - acc: 0.983 - ETA: 4:04 - loss: 0.1410 - acc: 0.983 - ETA: 4:03 - loss: 0.1410 - acc: 0.983 - ETA: 4:01 - loss: 0.1412 - acc: 0.983 - ETA: 4:00 - loss: 0.1413 - acc: 0.983 - ETA: 3:58 - loss: 0.1414 - acc: 0.983 - ETA: 3:56 - loss: 0.1412 - acc: 0.983 - ETA: 3:55 - loss: 0.1413 - acc: 0.983 - ETA: 3:53 - loss: 0.1410 - acc: 0.983 - ETA: 3:52 - loss: 0.1408 - acc: 0.983 - ETA: 3:50 - loss: 0.1405 - acc: 0.983 - ETA: 3:49 - loss: 0.1402 - acc: 0.983 - ETA: 3:48 - loss: 0.1403 - acc: 0.983 - ETA: 3:46 - loss: 0.1401 - acc: 0.983 - ETA: 3:44 - loss: 0.1399 - acc: 0.983 - ETA: 3:43 - loss: 0.1398 - acc: 0